# One-year binaries
A notebook to look at fitting issues for one-year binaries in ESA *Gaia* astrometric data.

## Authors:
- **Kareem El-Badry** (Caltech)
- **David W. Hogg** (NYU) (MPIA) (Flatiron)

## Dependencies:
- `numpy`
- `matplotlib`
- `gaiamock` <https://github.com/kareemelbadry/gaiamock>

## Projects:
- Compare likelihood ratios to marginal-likelihood ratios.
- Explore group averaging (objective Bayes).

In [ ]:
import gaiamock
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
c_funcs = gaiamock.read_in_C_functions()

# generate epoch astrometry for a random binary with Period = 500 d
t_ast_yr, psi, plx_factor, ast_obs, ast_err = gaiamock.predict_astrometry_luminous_binary(ra = 21.2, 
    dec=-16.4, parallax = 3.0, pmra = -7.5, pmdec = 13.2, m1 = 0.9, m2 = 0.65, period = 500, Tp = 293, ecc = 0.0, 
    omega = 2.0, inc = 1.5, w = 1.0, phot_g_mean_mag = 13.0, f = 0.2, data_release = 'dr3', c_funcs = c_funcs)

# plot it
plt.errorbar(t_ast_yr, ast_obs, yerr=ast_err, fmt='ko')
plt.xlabel('time (years)')
plt.ylabel(r'$\eta\,[\rm mas]$')

In [ ]:
# do a maximum-likelihood fit (fastest way to find the best solution)
res = gaiamock.fit_orbital_solution_nonlinear(t_ast_yr = t_ast_yr, psi = psi, plx_factor = plx_factor, 
            ast_obs = ast_obs, ast_err = ast_err, c_funcs = c_funcs)
gaiamock.plot_residuals(t_ast_yr, psi, plx_factor, ast_obs, ast_err, theta_array=res, c_funcs=c_funcs)


In [ ]:
# to make a corner plot, draw 1e7 samples and use rejection sampling to discard the ones with low relative likelihoods


samples = gaiamock.generate_prior_samples(N_samps = int(1e7), P_range = [100, 1000])
L = gaiamock.get_astrometric_likelihoods(t_ast_yr = t_ast_yr, psi = psi, plx_factor = plx_factor, 
                                         ast_obs = ast_obs, ast_err = ast_err, samples = samples)


r = np.random.uniform(0, np.max(L), len(L))
keep = L > r
 
keep_P, keep_ecc, keep_phi_p = np.array(samples).T[keep].T
keep_a0, keep_inc, keep_w, keep_omega, keep_plx = [], [], [], [], []
for i in range(len(keep_P)):

    period, phi_p, ecc = keep_P[i], keep_phi_p[i], keep_ecc[i]
    chi2, mu_linear = gaiamock.get_astrometric_chi2(t_ast_yr = t_ast_yr, psi = psi, plx_factor = plx_factor, 
        ast_obs = ast_obs, ast_err = ast_err, P = period, phi_p = phi_p, ecc = ecc, c_funcs=c_funcs)
    ra_off, pmra, dec_off, pmdec, plx, B, G, A, F = mu_linear
    a0_mas, inc, w, omega = gaiamock.get_Campbell_elements(A=A, B=B, F=F, G=G)
    keep_a0.append(a0_mas)
    keep_inc.append(inc)
    keep_w.append(w)
    keep_omega.append(omega)
    keep_plx.append(plx)

import corner
flatchain = np.vstack([keep_P, keep_ecc, keep_a0, keep_inc,  keep_omega, keep_plx ]).T
_ = corner.corner(flatchain, labels=[r'$P_{\rm orb}\,[\rm days]$', r'$\rm ecc$', r'$a_0\,[\rm mas]$', r'$\rm inc$', r'$\Omega$', r'$\varpi\,[\rm mas]$'], plot_contours=False, plot_density=False, show_titles=True)


In [ ]:
# now do the same thing for a 1 year period
t_ast_yr, psi, plx_factor, ast_obs, ast_err = gaiamock.predict_astrometry_luminous_binary(ra = 21.2, 
    dec=-16.4, parallax = 3.0, pmra = -7.5, pmdec = 13.2, m1 = 0.9, m2 = 0.65, period = 365.25, Tp = 293, ecc = 0.0, omega = 2.0, inc = 1.5, w = 1.0, phot_g_mean_mag = 13.0, f = 0.2, data_release = 'dr3', c_funcs = c_funcs)

samples = gaiamock.generate_prior_samples(N_samps = int(1e7), P_range = [100, 1000])
L = gaiamock.get_astrometric_likelihoods(t_ast_yr = t_ast_yr, psi = psi, plx_factor = plx_factor, 
                                         ast_obs = ast_obs, ast_err = ast_err, samples = samples)

r = np.random.uniform(0, np.max(L), len(L))
keep = L > r
 
keep_P, keep_ecc, keep_phi_p = np.array(samples).T[keep].T
keep_a0, keep_inc, keep_w, keep_omega, keep_plx = [], [], [], [], []
for i in range(len(keep_P)):

    period, phi_p, ecc = keep_P[i], keep_phi_p[i], keep_ecc[i]
    chi2, mu_linear = gaiamock.get_astrometric_chi2(t_ast_yr = t_ast_yr, psi = psi, plx_factor = plx_factor, 
        ast_obs = ast_obs, ast_err = ast_err, P = period, phi_p = phi_p, ecc = ecc, c_funcs=c_funcs)
    ra_off, pmra, dec_off, pmdec, plx, B, G, A, F = mu_linear
    a0_mas, inc, w, omega = gaiamock.get_Campbell_elements(A=A, B=B, F=F, G=G)
    keep_a0.append(a0_mas)
    keep_inc.append(inc)
    keep_w.append(w)
    keep_omega.append(omega)
    keep_plx.append(plx)

import corner
flatchain = np.vstack([keep_P, keep_ecc, keep_a0, keep_inc,  keep_omega, keep_plx ]).T
_ = corner.corner(flatchain, labels=[r'$P_{\rm orb}\,[\rm days]$', r'$\rm ecc$', r'$a_0\,[\rm mas]$', r'$\rm inc$', r'$\Omega$', r'$\varpi\,[\rm mas]$'], plot_contours=False, plot_density=False, show_titles=True)

In [ ]:
# now do the same thing for a 1 year period but eccentric
t_ast_yr, psi, plx_factor, ast_obs, ast_err = gaiamock.predict_astrometry_luminous_binary(ra = 21.2, 
    dec=-16.4, parallax = 3.0, pmra = -7.5, pmdec = 13.2, m1 = 0.9, m2 = 0.65, period = 365.25, Tp = 293, ecc = 0.5, omega = 2.0, inc = 1.5, w = 1.0, phot_g_mean_mag = 13.0, f = 0.2, data_release = 'dr3', c_funcs = c_funcs)

samples = gaiamock.generate_prior_samples(N_samps = int(1e7), P_range = [100, 1000])
L = gaiamock.get_astrometric_likelihoods(t_ast_yr = t_ast_yr, psi = psi, plx_factor = plx_factor, 
                                         ast_obs = ast_obs, ast_err = ast_err, samples = samples)

r = np.random.uniform(0, np.max(L), len(L))
keep = L > r
 
keep_P, keep_ecc, keep_phi_p = np.array(samples).T[keep].T
keep_a0, keep_inc, keep_w, keep_omega, keep_plx = [], [], [], [], []
for i in range(len(keep_P)):

    period, phi_p, ecc = keep_P[i], keep_phi_p[i], keep_ecc[i]
    chi2, mu_linear = gaiamock.get_astrometric_chi2(t_ast_yr = t_ast_yr, psi = psi, plx_factor = plx_factor, 
        ast_obs = ast_obs, ast_err = ast_err, P = period, phi_p = phi_p, ecc = ecc, c_funcs=c_funcs)
    ra_off, pmra, dec_off, pmdec, plx, B, G, A, F = mu_linear
    a0_mas, inc, w, omega = gaiamock.get_Campbell_elements(A=A, B=B, F=F, G=G)
    keep_a0.append(a0_mas)
    keep_inc.append(inc)
    keep_w.append(w)
    keep_omega.append(omega)
    keep_plx.append(plx)

import corner
flatchain = np.vstack([keep_P, keep_ecc, keep_a0, keep_inc,  keep_omega, keep_plx ]).T
_ = corner.corner(flatchain, labels=[r'$P_{\rm orb}\,[\rm days]$', r'$\rm ecc$', r'$a_0\,[\rm mas]$', r'$\rm inc$', r'$\Omega$', r'$\varpi\,[\rm mas]$'], plot_contours=False, plot_density=False, show_titles=True)